## GRINDA - Graph-based Intelligence for Network Disruption and Analysis

### Justification: 
The striking name refers to “grinding down” (dismantling) and highlights the network's analytical and disruptive focus.

### Graph

In [1]:
import pandas as pd
import networkx as nx
import plotly.graph_objects as go
import pickle
import numpy as np
import warnings
warnings.filterwarnings('ignore')

In [2]:
%reload_ext watermark
%watermark -a "Alex S.O. Toledo"

Author: Alex S.O. Toledo



In [3]:
class NetworkDisruptor:
    def __init__(self, network_number):
        # Initialize the disruptor with the network number
        self.network_number = network_number
        self.graph_type = None
        self.channels = None
        self.model = None
        self.steps_to_disruption = None
        self.ranking_df = None
        self.edgelist_df = None
        self.G_full = None
        self.plotter = None

    def load_best_model_params(self, csv_path='data/best_models_by_network.csv'):
        # Load the best model parameters for the given network from CSV
        df = pd.read_csv(csv_path, sep=';')
        row = df[df['network'] == self.network_number].iloc[0]
        self.graph_type = row['graph']
        self.channels = int(row['channels'])
        self.model = row['model']
        self.steps_to_disruption = int(row['steps_to_disruption'])

    def load_ranking(self, pkl_path='data/gt_ranking.pkl'):
        # Load the ranking DataFrame for the selected network, channels, and model
        with open(pkl_path, 'rb') as f:
            ranking_tables = pickle.load(f)
        self.ranking_df = ranking_tables[self.network_number][self.channels][self.model]

    def load_edgelist(self):
        # Load the edgelist for the network from file
        edgelist_path = f'data/network_{self.network_number}.edgelist'
        self.edgelist_df = pd.read_csv(edgelist_path, delim_whitespace=True, header=None, names=['Source', 'Target', 'Attr'])
        self.edgelist_df = self.edgelist_df[['Source', 'Target']]

    def build_graph(self):
        # Build a directed graph from the edgelist DataFrame
        nodes_in_source = set(self.edgelist_df['Source'].dropna())
        nodes_in_target = set(self.edgelist_df['Target'].dropna())
        all_nodes = nodes_in_source.union(nodes_in_target)
        isolated_nodes = set()
        # Find isolated nodes (nodes with only one side filled)
        if self.edgelist_df['Source'].isnull().any():
            isolated_nodes.update(self.edgelist_df.loc[self.edgelist_df['Source'].isnull(), 'Target'].dropna())
        if self.edgelist_df['Target'].isnull().any():
            isolated_nodes.update(self.edgelist_df.loc[self.edgelist_df['Target'].isnull(), 'Source'].dropna())
        all_nodes = all_nodes.union(isolated_nodes)
        G = nx.DiGraph()
        G.add_nodes_from(all_nodes)
        edges = self.edgelist_df.dropna(subset=['Source', 'Target']).values.tolist()
        G.add_edges_from(edges)
        self.G_full = G

    def print_components(self):
        # Print network statistics: number of nodes, nodes in largest component, components, and isolated nodes
        total_nodes = self.G_full.number_of_nodes()
        largest_component_nodes = max(nx.weakly_connected_components(self.G_full), key=len)
        num_largest_component_nodes = len(largest_component_nodes)
        componentes = total_nodes - num_largest_component_nodes
        #print(f"Total Nodes: {total_nodes}")
        print(f"Largest Component Nodes: {num_largest_component_nodes}")
        print(f"Componentes: {componentes}")
        print("Isolade Nodes:", list(nx.isolates(self.G_full)))

    def plot_graph(self, title="Network Graph"):
        # Plot the current state of the network in 3D
        if self.plotter is None:
            self.plotter = Network3DPlotter(None)
        self.plotter.G = self.G_full
        self.plotter.compute_layout(k=30, iterations=1000, seed=42)
        self.plotter.plot(title=title)

    def disrupt_network(self):
        # Remove nodes step by step according to the ranking until steps_to_disruption
        for step in range(self.steps_to_disruption):
            if step >= len(self.ranking_df):
                print("Fim do ranking.")
                break
            node_to_remove = self.ranking_df.iloc[step]
            if node_to_remove in self.G_full:
                self.G_full.remove_node(node_to_remove)
            print(f"Step {step+1}: Removed node {node_to_remove}")
            self.print_components()

class Network3DPlotter:
    def __init__(self, csv_path, source_col='Source', target_col='Target', directed=True):
        # Initialize the 3D plotter
        self.csv_path = csv_path
        self.source_col = source_col
        self.target_col = target_col
        self.directed = directed
        self.df = None
        self.G = None
        self.pos = None

    def compute_layout(self, k=30, iterations=1000, seed=42):
        # Compute the 3D layout for the graph nodes
        self.pos = nx.spring_layout(self.G, dim=3, seed=seed, k=k, iterations=iterations)

    def plot(self, node_color_rgba='rgba(142,202,230,0.6)', node_border_color='rgba(2,48,71,1)',
             node_size=20, node_font_size=13, node_opacity=0.7, title="Initial Network Graph"):
        # Plot the graph in 3D using Plotly
        Xn = [self.pos[k][0] for k in self.G.nodes()]
        Yn = [self.pos[k][1] for k in self.G.nodes()]
        Zn = [self.pos[k][2] for k in self.G.nodes()]
        labels = [str(n) for n in self.G.nodes()]
        Xe, Ye, Ze = [], [], []
        for e in self.G.edges():
            Xe += [self.pos[e[0]][0], self.pos[e[1]][0], None]
            Ye += [self.pos[e[0]][1], self.pos[e[1]][1], None]
            Ze += [self.pos[e[0]][2], self.pos[e[1]][2], None]
        fig = go.Figure()
        fig.add_trace(go.Scatter3d(
            x=Xe, y=Ye, z=Ze,
            mode='lines',
            line=dict(color='gray', width=2),
            hoverinfo='none'
        ))
        fig.add_trace(go.Scatter3d(
            x=Xn, y=Yn, z=Zn,
            mode='markers+text',
            marker=dict(
                symbol='circle',
                size=node_size,
                color=node_color_rgba,
                line=dict(color=node_border_color, width=3),
                opacity=node_opacity
            ),
            text=labels,
            textposition="middle center",
            textfont=dict(size=node_font_size, color='black'),
            hoverinfo='text'
        ))
        fig.update_layout(
            title=title,
            showlegend=False,
            scene=dict(
                xaxis=dict(showbackground=False, showticklabels=False, visible=False),
                yaxis=dict(showbackground=False, showticklabels=False, visible=False),
                zaxis=dict(showbackground=False, showticklabels=False, visible=False),
            ),
            margin=dict(l=0, r=0, b=0, t=40),
            width=1200,
            height=1000,
            paper_bgcolor='rgba(0,0,0,0)',
            plot_bgcolor='rgba(0,0,0,0)'
        )
        fig.show()

In [4]:
# Usage:
network_number = 53  # or any other network number
disruptor = NetworkDisruptor(network_number)
disruptor.load_best_model_params()   # Load best model parameters from CSV
disruptor.load_ranking()             # Load ranking from pickle file
disruptor.load_edgelist()            # Load edgelist from file
disruptor.build_graph()              # Build the networkx graph

# Print only the total number of nodes before disruption
print("Graph before disruption:")
print(f"Total Nodes: {disruptor.G_full.number_of_nodes()}")
disruptor.plot_graph(title="Full graph (before disruption)")

print("Graph after disruption:")
disruptor.disrupt_network()          # Disrupt the network step by step
disruptor.plot_graph(title="Graph after disruption")

Graph before disruption:
Total Nodes: 45


Graph after disruption:
Step 1: Removed node FJVL
Largest Component Nodes: 42
Componentes: 2
Isolade Nodes: []
Step 2: Removed node X4Q6
Largest Component Nodes: 40
Componentes: 3
Isolade Nodes: ['YFV7']
Step 3: Removed node H6GS
Largest Component Nodes: 38
Componentes: 4
Isolade Nodes: ['HGUT', 'YFV7']
Step 4: Removed node ANL3
Largest Component Nodes: 37
Componentes: 4
Isolade Nodes: ['HGUT', 'YFV7']
Step 5: Removed node Y1V5
Largest Component Nodes: 31
Componentes: 9
Isolade Nodes: ['HGUT', 'YFV7']
Step 6: Removed node JDH2
Largest Component Nodes: 30
Componentes: 9
Isolade Nodes: ['HGUT', 'YFV7']
Step 7: Removed node IBM9
Largest Component Nodes: 30
Componentes: 8
Isolade Nodes: ['HGUT', 'N6QV', 'YFV7']
Step 8: Removed node H2XI
Largest Component Nodes: 27
Componentes: 10
Isolade Nodes: ['HGUT', 'N6QV', 'X7QG', 'HCGF', 'YFV7']
Step 9: Removed node QRJ5
Largest Component Nodes: 22
Componentes: 14
Isolade Nodes: ['HGUT', 'SZZN', 'N6QV', 'DKN3', 'X7QG', 'HCGF', 'YFV7']
Step 10: Remove

## ///////////////////////////////////////////////////////////////////////////////////////

In [5]:
%watermark -v -m

Python implementation: CPython
Python version       : 3.12.7
IPython version      : 8.27.0

Compiler    : Clang 14.0.6 
OS          : Darwin
Release     : 24.5.0
Machine     : x86_64
Processor   : i386
CPU cores   : 10
Architecture: 64bit



In [6]:
%watermark --iversions

pandas  : 2.2.2
networkx: 3.3
numpy   : 1.26.4
plotly  : 5.24.1

